> This notebook has been tested on the SageMaker Studio *`Data Science 3.0`* kernel and ml.t3.medium instance.

## 0. Prerequisites
- For this hands-on exercise, the role (Role) executing the notebook must have the following permissions added.
    - AmazonOpenSearchServiceFullAccess
    - AmazonSSMFullAccess

## OpenSearch Client, Store Authentication Info in SSM

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import boto3
import uuid
import botocore
import time

In [4]:
from utils.ssm import parameter_store

In [5]:
DEV = True # If True, create as 1-AZ without standby, if False, create as 3-AZ with standby. For workshop purposes, it is recommended to set it to True to avoid excessive charges/resources.
VERSION = "2.11" # OpenSearch Version (e.g. 2.7 / 2.9 / 2.11)

In [6]:
opensearch_user_id = "<your id>" # ex) 'raguser'
opensearch_user_password = "<your password>" # ex) 'MarsEarth1!'

opensearch_user_id = "raguser"
opensearch_user_password = "MarsEarth1!"

In [7]:
# 0. Store OpenSearch authentication information in SSM

region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity()["Account"]
opensearch = boto3.client('opensearch', region)
rand_str = uuid.uuid4().hex[:8]
domain_name = f'rag-hol-{rand_str}'

cluster_config_prod = {
    'InstanceCount': 3,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': True,
    'DedicatedMasterEnabled': True,
    'MultiAZWithStandbyEnabled': True,
    'DedicatedMasterType': 'r6g.large.search',
    'DedicatedMasterCount': 3
}

cluster_config_dev = {
    'InstanceCount': 1,
    'InstanceType': 'r6g.large.search',
    'ZoneAwarenessEnabled': False,
    'DedicatedMasterEnabled': False,
}


ebs_options = {
    'EBSEnabled': True,
    'VolumeType': 'gp3',
    'VolumeSize': 100,
}

advanced_security_options = {
    'Enabled': True,
    'InternalUserDatabaseEnabled': True,
    'MasterUserOptions': {
        'MasterUserName': opensearch_user_id,
        'MasterUserPassword': opensearch_user_password
    }
}

ap = f'{{\"Version\":\"2012-10-17\",\"Statement\":[{{\"Effect\":\"Allow\",\"Principal\":{{\"AWS\":\"*\"}},\"Action\":\"es:*\",\"Resource\":\"arn:aws:es:{region}:{account_id}:domain\/{domain_name}\/*\"}}]}}'

if DEV:
    cluster_config = cluster_config_dev
else:
    cluster_config = cluster_config_prod

response = opensearch.create_domain(
    DomainName=domain_name,
    EngineVersion=f'OpenSearch_{VERSION}',
    ClusterConfig=cluster_config,
    AccessPolicies=ap,
    EBSOptions=ebs_options,
    AdvancedSecurityOptions=advanced_security_options,
    NodeToNodeEncryptionOptions={'Enabled': True},
    EncryptionAtRestOptions={'Enabled': True},
    DomainEndpointOptions={'EnforceHTTPS': True}
)

In [ ]:
%%time

# 1. Install OpenSearch

def wait_for_domain_creation(domain_name):
    try:
        response = opensearch.describe_domain(
            DomainName=domain_name
        )
        # Every 60 seconds, check whether the domain is processing.
        while 'Endpoint' not in response['DomainStatus']:
            print('Creating Opensearch domain...')
            time.sleep(60)
            response = opensearch.describe_domain(
                DomainName=domain_name)

        # Once we exit the loop, the domain is ready for ingestion.
        endpoint = response['DomainStatus']['Endpoint']
        print('Domain endpoint ready to receive data: ' + endpoint)
    except botocore.exceptions.ClientError as error:
        if error.response['Error']['Code'] == 'ResourceNotFoundException':
            print('Domain not found.')
        else:
            raise error

wait_for_domain_creation(domain_name)

response = opensearch.describe_domain(DomainName=domain_name)
opensearch_domain_endpoint = f"https://{response['DomainStatus']['Endpoint']}"

# 2. Store OpenSearch authentication information in SSM

region=boto3.Session().region_name
pm = parameter_store(region)

pm.put_params(
    key="opensearch_domain_endpoint",
    value=f'{opensearch_domain_endpoint}',
    overwrite=True,
    enc=False
)

pm.put_params(
    key="opensearch_user_id",
    value=f'{opensearch_user_id}',
    overwrite=True,
    enc=False
)

pm.put_params(
    key="opensearch_user_password",
    value=f'{opensearch_user_password}',
    overwrite=True,
    enc=True
)


Creating Opensearch domain...
Creating Opensearch domain...
Creating Opensearch domain...
Creating Opensearch domain...
Creating Opensearch domain...
Creating Opensearch domain...


In [ ]:
print (pm.get_params(key="opensearch_domain_endpoint", enc=False))
print (pm.get_params(key="opensearch_user_id", enc=False))
print (pm.get_params(key="opensearch_user_password", enc=True))